In [ ]:
# default_exp main_notebook

# option_cuda

> Subclass of option_base.  Python implementation of monte carlo path dependendent walk for European options.

This Notebook also shows how you can compile 

In [1]:
#hide
from nbdev.showdoc import *
def generate_python():
    from nbdev.export import notebook2script; notebook2script()
    
%load_ext autoreload
%autoreload 2

In [1]:
#export
import my_python_code.library_notebook as funcs


# Compile CUDA Kernel

In [2]:
# Compile if i needit

# Build both shared object and also executable.
def compile():
    !module load cuda; nvcc ../c_code/mc_option_v2_cachingload.cu -o mc_option_v2_cachingload.so -shared -Xcompiler -fPIC -lcurand 
    !module load cuda; nvcc ../c_code/mc_option.cu -o mc_option.so -shared -Xcompiler -fPIC -lcurand 
    !nvcc -o mc_option  ../c_code/mc_option.cu -lcurand 2> /dev/null
    # Print out the symbol table.  Useful for debugging ...
    !nm -CD mc_option.so | grep " T "
!pwd
compile()
!ls -lart mc_option.so

/home/dvanstee/projects/2021-03-montecarlo/nbs
../c_code/mc_option_v2_cachingload.cu(186): warning: argument is incompatible with corresponding format string conversion

../c_code/mc_option_v2_cachingload.cu(186): warning: argument is incompatible with corresponding format string conversion

000000000005ca7c T _fini
0000000000007650 T _init
0000000000009859 T main
0000000000008f96 T run_monte_carlo_cuda
0000000000008eef T set_device(_McSettings)
00000000000087dd T print_tsrow(int, _McSettings, float*)
0000000000008452 T check_curand(int)
0000000000009de2 T apply_strike_k(_McSettings, float*)
00000000000084e7 T print_settings(_McSettings)
0000000000008416 T check_cuda_call(int)
0000000000008a7c T init_gpu_memory(_McSettings, float**, float**, _McOption*)
0000000000009f51 T price_options_k(_McSettings, float*, _McOption)
0000000000009c8c T monte_carlo_rw_k(_McSettings, float*, float*)
0000000000008d8d T gpu_memory_cleanup(float*, float*)
0000000000008a3a T calculate_walk_step(_McSettings

In [3]:
# !./mc_option -h ----->>>>>  mc_option [-v] [-n num_trials] [-t time_steps] [-h]
!pwd
!./mc_option -n 1000000 -t 1000 -v


/home/dvanstee/projects/2021-03-montecarlo/nbs
Overriding num_trialsOverriding time_stepsVerbose mode


////PASS 1 //////
[set_device] : GPU Device 0: "Tesla V100S-PCIE-32GB" with compute capability 7.0
[set_device] : GPU Device = 0

[print_settings]:
 num_trials 1000000 
 time_steps 1000 
 block_size 1024 
 mu         0.000100 
 sigma      0.020000 
 initial_price 100.000000 
 strike     95.000000 
 rfr        0.010000 
 RNG seed   5 
 MC mode    0 (0=price,1=option)
 GPU Device 0 
 Verbose    1 

Projected CUDA Allocations
 Total Threads 1000000 
 Total Thread Blocks 977 
 Threads/Block 1024 
 CUDA Memory Footprint (rough) 2^29 * 4B
[run_monte_carlo_cuda] : /************* INIT MEMORY *****************
[run_monte_carlo_cuda] : /************* Returning Price Paths *****************
[run_monte_carlo_cuda] : /************* RUN MC PATHS *****************
[run_monte_carlo_cuda] : /************* RUN MC Pricing *****************
[run_monte_carlo_cuda] : ***** Runtime RNG/InitAry    0.250989 

In [4]:
#export
class _McSettings(ctypes.Structure):
    _fields_ = [('num_trials', ctypes.c_int),
                ('time_steps', ctypes.c_int),
                ('block_size', ctypes.c_int),
                ('mu', ctypes.c_float),
                ('sigma', ctypes.c_float),
                ('initial_price', ctypes.c_float),
                ('strike', ctypes.c_float),
                ('rfr', ctypes.c_float),
                ('verbose', ctypes.c_bool),
                ('seed', ctypes.c_int),
                ]


class OptionCuda(OptionBase):
    """
    Limitations : Different from pure python version.  I don't pass back all trials from device to host
      as thats inefficient and only used for random walk visuals.
    """

    # Required to access CUDA Kernel data structures

    def __init__(self, block_size=1024, **kwargs):
        super().__init__(**kwargs)
        self.block_size = block_size
        self.mode = "cuda"
        self.device = 0
        self.mc_so_path = "/home/dvanstee/projects/2021-03-montecarlo/nbs/mc_option.so"
        self.cfg = _McSettings(
            num_trials=self.num_trials,
            time_steps=self.time_steps,
            block_size=self.block_size,
            mu=self.mu,
            sigma=self.sigma,
            initial_price=self.initial_price,
            strike=self.strike,
            rfr=self.rfr,
            verbose=self.verbose,
            seed=self.seed)

    def print_mcSettings(self):
        for k, v in self.cfg._fields_:
            print("{:10s} {}".format(k, self.cfg.__getattribute__(k)))

    def run_monte_carlo(self, mc_mode: str = "options"):
        """
        Only mc_mode = options supported.
        """
        assert mc_mode == "options" or mc_mode == "prices"
        rv_tuple = (None, None)
        MonteCarlo = ctypes.cdll.LoadLibrary(self.mc_so_path)
        if mc_mode == "options":
            MonteCarlo.run_monte_carlo_cuda.restype = ndpointer(dtype=ctypes.c_float, shape=(2 * self.cfg.time_steps))
            mc_call_put_cuda = MonteCarlo.run_monte_carlo_cuda(self.cfg, mc_mode)
            call_option_prices = mc_call_put_cuda[0:self.cfg.time_steps]
            put_option_prices = mc_call_put_cuda[self.cfg.time_steps:2 * self.cfg.time_steps]
            rv_tuple = (call_option_prices, put_option_prices)
        else:
            MonteCarlo.run_monte_carlo_cuda.restype = ndpointer(dtype=ctypes.c_float,
                                                                shape=(self.cfg.time_steps * self.cfg.num_trials))
            mc_call_put_cuda = MonteCarlo.run_monte_carlo_cuda(self.cfg, mc_mode)
            rv_tuple = (mc_call_put_cuda, None)  # just price paths...

        return rv_tuple

    def get_option_expectation(self):
        """
        implicit_input : monte_carlo price paths (from self.run_monte_carlo)
        output : tuple(call_expectations, put_expectations)
          call_expectations in a numpy array of shape (timesteps, )
          put_expectations in a numpy array of shape (timesteps, )
        """

        option_prices = self.run_monte_carlo(mc_mode="options")
        return option_prices

In [5]:
mc_option = OptionCuda(num_trials=2**23,time_steps=365,strike=120,initial_price=90,rfr=0.00)
mc_option.seed=1
mc_option

num_trials 8388608
time_steps 365
mu         0.0001
sigma      0.025
initial_price 90
strike     120
rfr        0.0
verbose    False
seed       1
results    None
ver        0.1
block_size 1024
mode       cuda
device     0
mc_so_path /home/dvanstee/projects/2021-03-montecarlo/nbs/mc_option.so
cfg        <__main__._McSettings object at 0x2b5e584c5b40>

In [ ]:
with Timer(name="Time Cuda"):
    option_prices = mc_option.get_option_expectation()
    


In [ ]:
#option_prices.shape

In [ ]:
# Call / Put
try :
    assert option_prices[0].shape==(mc_option.time_steps,)
    assert option_prices[1].shape==(mc_option.time_steps,)
except :
    print(f"{option_prices[0].shape} {mc_option.time_steps} ")
    print(f"{option_prices[1].shape} {mc_option.time_steps} ")

# Visualize Some Results

In [ ]:
# plot_return_cdf(returns[128],returns[251],n_bins=33, labels=['day129','day252'])

In [ ]:
# plot_trajectories(returns)

In [ ]:
# get_return_percentiles(returns)

In [ ]:
plot_option_over_time(option_prices[0],title="Call Option Price vs Days to Expirations")
plot_option_over_time(option_prices[1],title="Put Option Price vs Days to Expirations")

In [ ]:
from nbdev.export import notebook2script; notebook2script()